## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,43.68,87,90,2.26,overcast clouds
1,1,Torrington,US,41.8007,-73.1212,94.41,43,90,1.99,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,42.12,70,100,9.75,overcast clouds
3,3,Vrangel,RU,42.7297,133.0839,54.79,83,13,4.92,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,76.41,69,27,15.28,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Vaini,TO,-21.2000,-175.2000,76.41,69,27,15.28,scattered clouds
7,7,Laguna,US,38.4210,-121.4238,89.46,38,1,4.00,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,79.70,64,71,8.37,broken clouds
18,18,Bassar,TG,9.2500,0.7833,81.93,59,97,4.34,overcast clouds
19,19,Hilo,US,19.7297,-155.0900,80.28,76,27,3.00,light rain
...,...,...,...,...,...,...,...,...,...,...
684,684,Daru,PG,-9.0763,143.2092,76.96,88,99,18.19,overcast clouds
686,686,Gizo,SB,-8.1030,156.8419,81.86,78,20,8.05,few clouds
688,688,Orocue,CO,4.7912,-71.3354,76.73,91,100,8.10,overcast clouds
689,689,Bangkalan,ID,-7.0455,112.7351,80.55,85,20,2.26,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Vaini,TO,-21.2000,-175.2000,76.41,69,27,15.28,scattered clouds
7,7,Laguna,US,38.4210,-121.4238,89.46,38,1,4.00,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,79.70,64,71,8.37,broken clouds
18,18,Bassar,TG,9.2500,0.7833,81.93,59,97,4.34,overcast clouds
19,19,Hilo,US,19.7297,-155.0900,80.28,76,27,3.00,light rain
...,...,...,...,...,...,...,...,...,...,...
684,684,Daru,PG,-9.0763,143.2092,76.96,88,99,18.19,overcast clouds
686,686,Gizo,SB,-8.1030,156.8419,81.86,78,20,8.05,few clouds
688,688,Orocue,CO,4.7912,-71.3354,76.73,91,100,8.10,overcast clouds
689,689,Bangkalan,ID,-7.0455,112.7351,80.55,85,20,2.26,few clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Vaini,TO,76.41,scattered clouds,-21.2000,-175.2000,
7,Laguna,US,89.46,clear sky,38.4210,-121.4238,
16,Atuona,PF,79.70,broken clouds,-9.8000,-139.0333,
18,Bassar,TG,81.93,overcast clouds,9.2500,0.7833,
19,Hilo,US,80.28,light rain,19.7297,-155.0900,
20,Alofi,NU,76.35,broken clouds,-19.0595,-169.9187,
24,Jamestown,US,84.29,scattered clouds,42.0970,-79.2353,
25,Mazamitla,MX,78.66,overcast clouds,19.9167,-103.0333,
27,Dingle,PH,75.67,light rain,10.9995,122.6711,
30,Vanimo,PG,76.91,light rain,-2.6741,141.3028,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Vaini,TO,76.41,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
7,Laguna,US,89.46,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
16,Atuona,PF,79.70,broken clouds,-9.8000,-139.0333,Villa Enata
18,Bassar,TG,81.93,overcast clouds,9.2500,0.7833,RGG
19,Hilo,US,80.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
684,Daru,PG,76.96,overcast clouds,-9.0763,143.2092,Daru Lodge
686,Gizo,SB,81.86,few clouds,-8.1030,156.8419,Imagination Island
688,Orocue,CO,76.73,overcast clouds,4.7912,-71.3354,Hotel Brisas de San Miguel
689,Bangkalan,ID,80.55,few clouds,-7.0455,112.7351,SPOT ON 2591 Griya Demangan Syariah


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))